In [1]:
import torch

print(torch.cuda.is_available())

True


In [15]:
import cv2
index = 1 # 从第一个设备开始检查
cap = cv2.VideoCapture(index, cv2.CAP_DSHOW)  # 使用CAP_DSHOW可以避免一些兼容性问题
while index<20:
    # 尝试打开当前索引对应的设备
    cap.open(index, cv2.CAP_DSHOW)
    
    # 检查是否成功打开设备
    
    if cap.isOpened():
        print(f"Camera found at index {index}")
        # 在这里可以做一些操作，比如获取帧并显示
        ret, frame = cap.read()
        if ret:
            cv2.imshow('Camera', frame)  # 显示相机捕获的第一帧
            cv2.waitKey(0)  # 等待键盘输入
            break
    else:
        print(f"No camera found at index {index}")
    cap.release()  # 释放设备
    index+=1

    index += 1  # 检查下一个设备


# cap = cv2.VideoCapture(7)  #调用摄像头‘0’一般是打开电脑自带摄像头，其他值是打开外部摄像头（只有一个摄像头的情况）
 
if False == cap.isOpened():
    print(0)
else:
    print(1)

No camera found at index 1
No camera found at index 3
No camera found at index 5
No camera found at index 7
No camera found at index 9
No camera found at index 11
No camera found at index 13
No camera found at index 15
No camera found at index 17
No camera found at index 19
0


In [16]:
from PIL import Image


img = Image.open(r'F:\work\python\clone\2d\Pytorch-UNet\data\imgs\sample\Image_20240622155143140_OUT.png')
print(img)

FileNotFoundError: [Errno 2] No such file or directory: 'F:\\work\\python\\clone\\2d\\Pytorch-UNet\\data\\imgs\\sample\\Image_20240622155143140_OUT.png'

In [ ]:
import cv2

img1=cv2.imread(r'F:\work\python\clone\2d\Pytorch-UNet\data\imgs\sample\barsample1_OUT.png')
print(img1)

In [1]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
import argparse
import logging
import os

import numpy as np
import torch
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms

from utils.data_loading import BasicDataset
from unet import UNet
from utils.utils import plot_img_and_mask

def predict_img(net, full_img, device, scale_factor=1, out_threshold=0.5):
    net.eval()
    img = torch.from_numpy(BasicDataset.preprocess(None, full_img, scale_factor, is_mask=False))
    img = img.unsqueeze(0)
    img = img.to(device=device, dtype=torch.float32)
    logging.info(f'Preprocessed image shape: {img.shape}')

    with torch.no_grad():
        output = net(img).cpu()
        output = F.interpolate(output, (full_img.size[1], full_img.size[0]), mode='bilinear')
        if net.n_classes > 1:
            mask = output.argmax(dim=1)
        else:
            mask = torch.sigmoid(output) > out_threshold
    logging.info(f'Predicted mask unique values: {torch.unique(mask)}')
    return mask[0].long().squeeze().numpy()

def get_args():
    parser = argparse.ArgumentParser(description='Predict mask from input images')
    parser.add_argument('--model', '-m', default='2d/Pytorch-UNet/model/checkpoint_epoch10.pth', metavar='FILE',
                        help='Specify the file in which the model is stored')
    parser.add_argument('--input', '-i', metavar='INPUT', nargs='+', help='Filenames of input images')
    parser.add_argument('--output', '-o', metavar='OUTPUT', nargs='+', help='Filenames of output images')
    parser.add_argument('--viz', '-v', action='store_true',
                        help='Visualize the images as they are processed')
    parser.add_argument('--no-save', '-n', action='store_true', help='Do not save the output mask')
    parser.add_argument('--mask-threshold', '-t', type=float, default=0.5,
                        help='Minimum probability value to consider a mask pixel white')
    parser.add_argument('--scale', '-s', type=float, default=0.5,
                        help='Scale factor for the input images')
    parser.add_argument('--bilinear', action='store_true', default=False, help='Use bilinear upsampling')
    parser.add_argument('--classes', '-c', type=int, default=2, help='Number of classes')

    args = parser.parse_args()

    if args.input is None:
        args.input = [r'F:\work\python\clone\2d\Pytorch-UNet\data\imgs\barsample.jpg']

    return args

def get_output_filenames(args):
    def _generate_name(fn):
        return f'{os.path.splitext(fn)[0]}_OUT.png'
    return args.output or list(map(_generate_name, args.input))

def mask_to_image(mask: np.ndarray, mask_values):
    logging.info(f'Mask shape: {mask.shape}, unique values: {np.unique(mask)}')
    if isinstance(mask_values[0], list):
        out = np.zeros((mask.shape[-2], mask.shape[-1], len(mask_values[0])), dtype=np.uint8)
    elif mask_values == [0, 1]:
        out = np.zeros((mask.shape[-2], mask.shape[-1]), dtype=bool)
    else:
        out = np.zeros((mask.shape[-2], mask.shape[-1]), dtype=np.uint8)

    if mask.ndim == 3:
        mask = np.argmax(mask, axis=0)

    for i, v in enumerate(mask_values):
        out[mask == i] = v

    return Image.fromarray(out)

if __name__ == '__main__':
    args = get_args()

    img_dir = r"F:\work\python\clone\2d\Pytorch-UNet\data\imgs\sample"
    fname = os.listdir(img_dir)
    for name in fname:
        fname2 = os.path.join(img_dir, name)
        if args.input is None:
            args.input = [fname2]
        else:
            args.input.append(fname2)

    logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

    in_files = args.input
    out_files = get_output_filenames(args)

    net = UNet(n_channels=3, n_classes=args.classes, bilinear=args.bilinear)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    logging.info(f'Loading model {args.model}')
    logging.info(f'Using device {device}')

    net.to(device=device)
    state_dict = torch.load(args.model, map_location=device)
    logging.info(f'Model loaded with parameters: {list(net.parameters())[0].data}')
    mask_values = state_dict.pop('mask_values', [0, 1])
    net.load_state_dict(state_dict)

    logging.info('Model loaded!')

    for i, filename in enumerate(in_files):
        logging.info(f'Predicting image {filename} ...')
        img = Image.open(filename)

        mask = predict_img(net=net,
                           full_img=img,
                           scale_factor=args.scale,
                           out_threshold=args.mask_threshold,
                           device=device)



usage: ipykernel_launcher.py [-h] [--model FILE] [--input INPUT [INPUT ...]]
                             [--output OUTPUT [OUTPUT ...]] [--viz]
                             [--no-save] [--mask-threshold MASK_THRESHOLD]
                             [--scale SCALE] [--bilinear] [--classes CLASSES]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\Liminghui\AppData\Roaming\jupyter\runtime\kernel-b15d4986-7d81-481e-a485-73882b6175e1.json


SystemExit: 2

F:\work\environment\envs\bar\lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [5]:
from PIL import Image
import numpy as np
import os

def get_max_gray_value(image_path):
    # 打开图像
    img = Image.open(image_path).convert('L')  # 转换为灰度图像

    # 转换为numpy数组
    img_array = np.array(img)

    # 获取最高灰度值
    max_gray_value = img_array.max()
    print(f"最高灰度值: {max_gray_value}")

    return img_array, img

def replace_gray_value(img_array, old_value, new_value):
    # 将灰度值为old_value的像素转换为new_value
    img_array[img_array == old_value] = new_value

    # 转换为图像
    new_img = Image.fromarray(img_array)

    return new_img

if __name__ == "__main__":
    # 替换为您的图片路径
    # image_path = 'path_to_your_image.png'
    image_path = r"F:\work\dataset\rebar2D\train2\mask"
    output_dir = r"F:\work\dataset\rebar2D\train2\TEMP"
    for filename in os.listdir(image_path):
        if filename.endswith(".png"):
            input_path = os.path.join(image_path, filename)
            # 获取最高灰度值并返回图像数组
            img_array, img = get_max_gray_value(input_path)

            # 将灰度值为1的像素转换为255
            new_img = replace_gray_value(img_array, 1, 255)

            # 保存新图像
            new_image_path = os.path.join(output_dir, filename)
            new_img.save(new_image_path)
            print(f"新图像已保存到: {new_image_path}")
    # # 获取最高灰度值并返回图像数组
    # img_array, img = get_max_gray_value("/kaggle/input/rebar2d/train2/mask/Image_20240622152006003_mask.png")
    # 
    # # 将灰度值为1的像素转换为255
    # new_img = replace_gray_value(img_array, 1, 255)
    # 
    # # 保存新图像
    # new_image_path = 'path_to_save_new_image.png'
    # new_img.save(new_image_path)
    # print(f"新图像已保存到: {new_image_path}")


最高灰度值: 1
新图像已保存到: F:\work\dataset\rebar2D\train2\TEMP\Image_20240622152006003_mask.png
最高灰度值: 1
新图像已保存到: F:\work\dataset\rebar2D\train2\TEMP\Image_20240622152022155_mask.png
最高灰度值: 1
新图像已保存到: F:\work\dataset\rebar2D\train2\TEMP\Image_20240622152027051_mask.png
最高灰度值: 1
新图像已保存到: F:\work\dataset\rebar2D\train2\TEMP\Image_20240622152032646_mask.png
最高灰度值: 1
新图像已保存到: F:\work\dataset\rebar2D\train2\TEMP\Image_20240622152038023_mask.png
最高灰度值: 1
新图像已保存到: F:\work\dataset\rebar2D\train2\TEMP\Image_20240622152043066_mask.png
最高灰度值: 1
新图像已保存到: F:\work\dataset\rebar2D\train2\TEMP\Image_20240622152049220_mask.png
最高灰度值: 1
新图像已保存到: F:\work\dataset\rebar2D\train2\TEMP\Image_20240622152054365_mask.png
最高灰度值: 1
新图像已保存到: F:\work\dataset\rebar2D\train2\TEMP\Image_20240622152100108_mask.png
最高灰度值: 1
新图像已保存到: F:\work\dataset\rebar2D\train2\TEMP\Image_20240622152105224_mask.png
最高灰度值: 1
新图像已保存到: F:\work\dataset\rebar2D\train2\TEMP\Image_20240622152110857_mask.png
最高灰度值: 1
新图像已保存到: F:\work\dataset\rebar2D\t

In [ ]:
import os
import json
import numpy as np
import cv2
from labelme import utils
from PIL import Image

def labelme_to_yolov8(labelme_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(os.path.join(output_dir, "images"), exist_ok=True)
    os.makedirs(os.path.join(output_dir, "labels"), exist_ok=True)
    os.makedirs(os.path.join(output_dir, "masks"), exist_ok=True)

    for filename in os.listdir(labelme_dir):
        if filename.endswith(".json"):
            json_path = os.path.join(labelme_dir, filename)
            with open(json_path) as f:
                data = json.load(f)
            
            image_data = utils.img_b64_to_arr(data['imageData'])
            img = Image.fromarray(image_data)
            img.save(os.path.join(output_dir, "images", filename.replace(".json", ".jpg")))

            shapes = data['shapes']
            mask = np.zeros(image_data.shape[:2], dtype=np.uint8)

            for shape in shapes:
                points = np.array(shape['points'], dtype=np.int32)
                cv2.fillPoly(mask, [points], color=1)
            
            mask_img = Image.fromarray(mask * 255)
            mask_img.save(os.path.join(output_dir, "masks", filename.replace(".json", ".png")))

            label_path = os.path.join(output_dir, "labels", filename.replace(".json", ".txt"))
            with open(label_path, 'w') as f:
                f.write(f"{filename.replace('.json', '')} {mask_path.replace('.json', '.png')}\n")

# 使用示例
labelme_dir = "path/to/labelme/json"
output_dir = "path/to/yolov8/dataset"
labelme_to_yolov8(labelme_dir, output_dir)


In [3]:
import os  # 导入操作系统模块
import json  # 导入JSON模块

def convert_json_to_txt_with_normalization_and_mapping(labelme_dir, output_dir, label_mapping):
    os.makedirs(output_dir, exist_ok=True)  # 创建输出目录，如果不存在则创建
    
    json_files = [f for f in os.listdir(labelme_dir) if f.endswith('.json')]  # 获取目录下所有JSON文件的列表
    
    for filename in json_files:  # 遍历每个JSON文件
        json_path = os.path.join(labelme_dir, filename)  # 构建JSON文件的完整路径
        txt_filename = filename.replace('.json', '.txt')  # 构建输出TXT文件的文件名
        txt_path = os.path.join(output_dir, txt_filename)  # 构建输出TXT文件的完整路径
        
        with open(json_path) as f:  # 打开JSON文件
            data = json.load(f)  # 加载JSON数据
        
        image_height = data['imageHeight']  # 获取图像的高度
        image_width = data['imageWidth']  # 获取图像的宽度
        
        with open(txt_path, 'w') as f:  # 打开（或创建）TXT文件以写入模式
            for shape in data['shapes']:  # 遍历每个形状
                label = shape['label']  # 获取形状的标签
                points = shape['points']  # 获取形状的坐标点
                normalized_points = [(x / image_width, y / image_height) for x, y in points]  # 归一化坐标点
                mapped_label = label_mapping.get(label, -1)  # 获取标签的映射值，默认为-1
                
                # 将标签和所有归一化后的坐标点写入TXT文件，一行表示一个实例
                points_str = " ".join([f"{x:.6f} {y:.6f}" for x, y in normalized_points])  # 将坐标点转换为字符串
                f.write(f"{mapped_label} {points_str}\n")  # 写入TXT文件

# 使用示例
labelme_dir = r"F:\work\python\clone\dataset\rebar2d\labelme_jsons"  # 指定Labelme JSON文件所在目录
output_dir = r"F:\work\python\clone\dataset\rebar2d\yololabel"  # 指定输出的TXT文件目录
label_mapping = {  # 标签映射字典
    "rebar": 0
    # 添加更多标签映射
}
convert_json_to_txt_with_normalization_and_mapping(labelme_dir, output_dir, label_mapping)  # 调用函数进行转换


In [5]:
##划分数据集
import os
import shutil
from sklearn.model_selection import train_test_split

# 定义函数以创建文件夹
def make_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

# 定义数据集划分函数
def split_dataset(image_dir, label_dir, output_dir, test_size=0.2, val_size=0.1, test_split=False):
    # 创建输出目录
    make_dir(output_dir)
    make_dir(os.path.join(output_dir, 'train'))
    make_dir(os.path.join(output_dir, 'val'))
    
    if test_split:
        make_dir(os.path.join(output_dir, 'test'))
    
    # 获取所有图像文件名
    images = os.listdir(image_dir)
    labels = os.listdir(label_dir)
    
    # 将文件名排序，以确保图像和标签对齐
    images.sort()
    labels.sort()
    
    # 划分训练集和临时集（包含验证集和可能的测试集）
    train_images, temp_images, train_labels, temp_labels = train_test_split(images, labels, test_size=(val_size + test_size))
    
    # 划分验证集和测试集
    if test_split:
        val_images, test_images, val_labels, test_labels = train_test_split(temp_images, temp_labels, test_size=(test_size / (val_size + test_size)))
    else:
        val_images, val_labels = temp_images, temp_labels

    # 定义复制文件的函数
    def copy_files(file_list, source_dir, dest_dir):
        for file in file_list:
            shutil.copy(os.path.join(source_dir, file), os.path.join(dest_dir, file))
    
    # 复制训练集文件
    copy_files(train_images, image_dir, os.path.join(output_dir, 'train'))
    copy_files(train_labels, label_dir, os.path.join(output_dir, 'train'))
    
    # 复制验证集文件
    copy_files(val_images, image_dir, os.path.join(output_dir, 'val'))
    copy_files(val_labels, label_dir, os.path.join(output_dir, 'val'))

    # 如果需要，复制测试集文件
    if test_split:
        copy_files(test_images, image_dir, os.path.join(output_dir, 'test'))
        copy_files(test_labels, label_dir, os.path.join(output_dir, 'test'))

# 示例使用
# input_dir = r"F:\work\python\clone\dataset\rebar2d\images"  
# output_dir = r"F:\work\python\clone\2d\ultralytics\dataset\rebar2d"  
image_dir = r"F:\work\python\clone\dataset\rebar2d\images"     # 图像目录路径
label_dir = r'F:\work\python\clone\dataset\rebar2d\yololabel'   # 标签目录路径
output_dir =  r"F:\work\python\clone\2d\ultralytics\dataset\rebar2d"    # 输出目录路径

split_dataset(image_dir, label_dir, output_dir, test_size=0.1, val_size=0, test_split=False)


In [ ]:
import os
import random
import cv2
import numpy as np
from albumentations import (HorizontalFlip, VerticalFlip, RandomRotate90, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue, RandomBrightnessContrast, GaussNoise, Compose)
from albumentations.core.serialization import save, load

# 定义数据增强的函数
def augment_data(image, mask):
    # 定义增强变换
    aug = Compose([
        HorizontalFlip(p=0.5),  # 随机水平翻转
        VerticalFlip(p=0.5),    # 随机垂直翻转
        RandomRotate90(p=0.5),  # 随机旋转90度
        ShiftScaleRotate(scale_limit=0.2, rotate_limit=45, shift_limit=0.1, p=0.5, border_mode=cv2.BORDER_REFLECT),
        Blur(blur_limit=3, p=0.2),  # 模糊
        OpticalDistortion(distort_limit=0.3, p=0.5),
        GridDistortion(p=0.5),
        HueSaturationValue(p=0.5),  # 色调饱和度变化
        RandomBrightnessContrast(p=0.5),  # 随机亮度对比度
        GaussNoise(p=0.2)  # 高斯噪声
    ])
    
    # 应用增强变换
    augmented = aug(image=image, mask=mask)
    return augmented['image'], augmented['mask']

# 定义处理单个文件的函数
def process_file(image_path, mask_path, output_image_dir, output_mask_dir, num_augments=5):
    # 读取图像和标签
    image = cv2.imread(image_path)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # 标签通常是灰度图
    
    for i in range(num_augments):
        # 增强数据
        aug_image, aug_mask = augment_data(image, mask)
        
        # 生成文件名
        base_name = os.path.splitext(os.path.basename(image_path))[0]
        image_save_path = os.path.join(output_image_dir, f"{base_name}_aug_{i}.png")
        mask_save_path = os.path.join(output_mask_dir, f"{base_name}_aug_{i}.png")
        
        # 保存增强后的图像和标签
        cv2.imwrite(image_save_path, aug_image)
        cv2.imwrite(mask_save_path, aug_mask)

# 定义处理整个目录的函数
def process_directory(image_dir, mask_dir, output_image_dir, output_mask_dir, num_augments=5):
    # 创建输出目录
    os.makedirs(output_image_dir, exist_ok=True)
    os.makedirs(output_mask_dir, exist_ok=True)
    
    # 获取所有图像文件
    image_files = os.listdir(image_dir)
    
    for image_file in image_files:
        image_path = os.path.join(image_dir, image_file)
        mask_path = os.path.join(mask_dir, image_file)  # 假设图像和标签文件名相同
        
        if os.path.exists(mask_path):
            process_file(image_path, mask_path, output_image_dir, output_mask_dir, num_augments)

# 示例使用
image_dir = 'path/to/images'        # 原始图像目录
label_dir = 'path/to/labels'        # 原始标签目录
output_image_dir = 'path/to/augmented_images'  # 增强后的图像目录
output_label_dir = 'path/to/augmented_labels'  # 增强后的标签目录

# 处理整个目录
process_directory(image_dir, label_dir, output_image_dir, output_label_dir, num_augments=5)


In [2]:
import cv2  # 导入OpenCV库
import json  # 导入JSON库
import numpy as np  # 导入NumPy库

def generate_json_from_mask(mask_path, json_path, label):
    # 读取掩码图像
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    # 查找图像中的轮廓
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # 确保找到至少一个轮廓
    if len(contours) == 0:
        raise ValueError("No contours found in mask image")

    # 只取最大的轮廓
    contour = max(contours, key=cv2.contourArea)
    
    # 将轮廓坐标转换为列表
    contour_list = contour.squeeze().tolist()
    
    # 检查并处理单点情况
    if isinstance(contour_list[0], int):
        contour_list = [contour_list]
    
    # 创建标签数据结构
    data = {
        "label": label,
        "points": contour_list
    }
    
    # 将数据写入JSON文件
    with open(json_path, 'w') as json_file:
        json.dump(data, json_file, indent=4)

# 示例使用
mask_path = r"F:\work\dataset\rebar2D\train\TEMP\Image_20240622152022155.png"  # 掩码图像路径
json_path = r"F:\work\dataset\rebar2D\train\jsontest\jsontest.json"  # 输出JSON文件路径
label = 'rebar'  # 目标标签

# 调用函数生成JSON文件
generate_json_from_mask(mask_path, json_path, label)


In [4]:
import cv2  # 导入OpenCV库
import json  # 导入JSON库
import numpy as np  # 导入NumPy库
import os  # 导入os库，用于文件和目录操作

def generate_json_from_mask(mask_path, json_path, label):
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # 读取掩码图像
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  # 查找图像中的轮廓

    if len(contours) == 0:  # 确保找到至少一个轮廓
        raise ValueError(f"No contours found in {mask_path}")

    contour = max(contours, key=cv2.contourArea)  # 只取最大的轮廓
    contour_list = contour.squeeze().tolist()  # 将轮廓坐标转换为列表

    if isinstance(contour_list[0], int):  # 检查并处理单点情况
        contour_list = [contour_list]

    # 创建标签数据结构
    data = {
        "shapes": [
            {
                "label": label,
                "points": contour_list,
                "group_id": None,
                "description": "",
                "shape_type": "polygon",
                "flags": {},
                "mask": None
            }
        ],
        "imagePath": mask_path,
        "imageData": None,
        "imageHeight": mask.shape[0],
        "imageWidth": mask.shape[1]
    }

    with open(json_path, 'w') as json_file:  # 将数据写入JSON文件
        json.dump(data, json_file, indent=4)

def process_masks_in_directory(directory, label):
    for filename in os.listdir(directory):  # 遍历目录中的所有文件
        if filename.endswith('.png'):  # 只处理PNG文件
            mask_path = os.path.join(directory, filename)  # 构建掩码图像路径
            json_path = os.path.splitext(mask_path)[0] + '.json'  # 构建JSON文件路径
            generate_json_from_mask(mask_path, json_path, label)  # 调用函数生成JSON文件

# 示例使用
directory = r"F:\work\dataset\rebar2D\train\TEMP"   # 掩码图像文件夹路径
# mask_path = r"F:\work\dataset\rebar2D\train\TEMP\Image_20240622152022155.png"  # 掩码图像路径
# json_path = r"F:\work\dataset\rebar2D\train\jsontest\jsontest.json"  # 输出JSON文件路径
label = 'rebar'  # 目标标签

process_masks_in_directory(directory, label)  # 处理目录中的所有掩码文件


In [5]:
import cv2  # 导入OpenCV库，用于图像处理
import numpy as np  # 导入NumPy库，用于数值计算
import albumentations as A  # 导入Albumentations库，用于数据增强
import os  # 导入os模块，用于文件和目录操作

def augment_image(image, mask):
    """
    对输入的图像和掩码进行数据增强
    :param image: 输入图像（H, W, C）
    :param mask: 输入掩码（H, W）
    :return: 增强后的图像和掩码
    """
    # 定义数据增强流水线
    transform = A.Compose([
        A.HorizontalFlip(p=0.5),  # 水平翻转，概率为0.5
        A.VerticalFlip(p=0.5),  # 垂直翻转，概率为0.5
        A.RandomRotate90(p=0.5),  # 随机旋转90度，概率为0.5
        A.Transpose(p=0.5),  # 转置，概率为0.5
        A.RandomBrightnessContrast(p=0.2),  # 随机亮度和对比度调整，概率为0.2
        A.ElasticTransform(p=0.2),  # 弹性变换，概率为0.2
        A.GridDistortion(p=0.2),  # 网格扭曲，概率为0.2
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=45, p=0.2),  # 平移、缩放和旋转，概率为0.2
        A.RandomCrop(height=256, width=256, p=1.0)  # 随机裁剪到256x256，概率为1.0
    ])
    
    # 应用数据增强到图像和掩码
    augmented = transform(image=image, mask=mask)
    image_aug = augmented['image']  # 增强后的图像
    mask_aug = augmented['mask']  # 增强后的掩码
    
    return image_aug, mask_aug  # 返回增强后的图像和掩码

def process_folder(input_image_folder, input_mask_folder, output_image_folder, output_mask_folder):
    """
    处理文件夹中的所有图像和掩码
    :param input_image_folder: 输入图像文件夹路径
    :param input_mask_folder: 输入掩码文件夹路径
    :param output_image_folder: 输出图像文件夹路径
    :param output_mask_folder: 输出掩码文件夹路径
    """
    if not os.path.exists(output_image_folder):
        os.makedirs(output_image_folder)  # 如果输出图像文件夹不存在，创建它
    if not os.path.exists(output_mask_folder):
        os.makedirs(output_mask_folder)  # 如果输出掩码文件夹不存在，创建它

    image_files = os.listdir(input_image_folder)  # 获取输入图像文件夹中的所有文件名

    for image_file in image_files:
        if image_file.endswith('.jpg'):
            image_file = os.path.join(image_file[:-4] + '.png')
        
        
        
        
        image_path = os.path.join(input_image_folder, image_file)  # 构建图像文件的完整路径
        mask_path = os.path.join(input_mask_folder, image_file)  # 构建对应掩码文件的完整路径
        
        if not os.path.exists(mask_path):
            print(f"掩码文件 {mask_path} 不存在，跳过。")
            continue
        
        image = cv2.imread(image_path)  # 读取图像
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # 读取掩码（灰度图像）
        
        image_aug, mask_aug = augment_image(image, mask)  # 对图像和掩码进行数据增强

        # 构建增强后图像和掩码的保存路径
        output_image_path = os.path.join(output_image_folder, image_file)
        output_mask_path = os.path.join(output_mask_folder, image_file)
        
        cv2.imwrite(output_image_path, image_aug)  # 保存增强后的图像
        cv2.imwrite(output_mask_path, mask_aug)  # 保存增强后的掩码

# 示例用法
if __name__ == "__main__":
    Dataset_Path = r"F:\work\dataset\rebar2D\train\label"  # 数据集路径
    img_path = r"F:\work\dataset\rebar2D\train\img"
    labelme_json_path = r"F:\work\dataset\rebar2D\train\label"
    mask_path=r"F:\work\dataset\rebar2D\train\mask" # 掩码图像文件夹路径
    
    input_image_folder = img_path  # 输入图像文件夹路径
    input_mask_folder = mask_path  # 输入掩码文件夹路径
    output_image_folder = r"F:\work\dataset\rebar2D\train\img1"  # 输出图像文件夹路径
    output_mask_folder = r"F:\work\dataset\rebar2D\train\mask1"  # 输出掩码文件夹路径

    process_folder(input_image_folder, input_mask_folder, output_image_folder, output_mask_folder)  # 处理文件夹中的所有图像和掩码


TypeError: image must be numpy array type

In [8]:
import cv2
import numpy as np
import albumentations as A
import os
import json

def augment_image(image, mask):
    transform = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.Transpose(p=0.5),
        A.RandomBrightnessContrast(p=0.2),
        A.ElasticTransform(p=0.2),
        A.GridDistortion(p=0.2),
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=45, p=0.2),
        # A.RandomCrop(height=256, width=256, p=1.0)
    ])
    
    augmented = transform(image=image, mask=mask)
    image_aug = augmented['image']
    mask_aug = augmented['mask']
    
    return image_aug, mask_aug

def read_mask(mask_path, label_format):
    if label_format == "mask":
        mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)
    elif label_format == "json":
        with open(mask_path, 'r') as f:
            annotations = json.load(f)
            mask = np.zeros((annotations['imageHeight'], annotations['imageWidth']), dtype=np.uint8)
            for shape in annotations['shapes']:
                points = np.array(shape['points'], dtype=np.int32)
                cv2.fillPoly(mask, [points], color=(255))
    else:
        raise ValueError("Unsupported label format. Use 'mask' or 'json'.")
    return mask

def process_folder(input_image_folder, input_mask_folder, output_image_folder, output_mask_folder, label_format="mask"):
    if not os.path.exists(output_image_folder):
        os.makedirs(output_image_folder)
    if not os.path.exists(output_mask_folder):
        os.makedirs(output_mask_folder)

    image_files = os.listdir(input_image_folder)

    for image_file in image_files:
        image_path = os.path.join(input_image_folder, image_file)
        
        mask_file = image_file.replace('.jpg', '.png') if label_format == "mask" else image_file.replace('.jpg', '.json')
        mask_path = os.path.join(input_mask_folder, mask_file)
        
        if not os.path.exists(mask_path):
            print(f"Label file {mask_path} does not exist, skipping.")
            continue
        
        image = cv2.imread(image_path)
        mask = read_mask(mask_path, label_format)
        
        image_aug, mask_aug = augment_image(image, mask)
        
        output_image_path = os.path.join(output_image_folder, image_file)
        output_mask_path = os.path.join(output_mask_folder, mask_file)
        
        cv2.imwrite(output_image_path, image_aug)
        cv2.imwrite(output_mask_path, mask_aug)

if __name__ == "__main__":
    Dataset_Path = r"F:\work\dataset\rebar2D\train\label"  # 数据集路径
    img_path = r"F:\work\dataset\rebar2D\train\img"
    labelme_json_path = r"F:\work\dataset\rebar2D\train\label"
    mask_path=r"F:\work\dataset\rebar2D\train\mask" # 掩码图像文件夹路径
    
    input_image_folder = img_path  # 输入图像文件夹路径
    input_mask_folder = mask_path  # 输入掩码文件夹路径
    output_image_folder = r"F:\work\dataset\rebar2D\train\img1"  # 输出图像文件夹路径
    output_mask_folder = r"F:\work\dataset\rebar2D\train\mask1"  # 输出掩码文件夹路径
    

    label_format = "mask"  # or "json"
    
    process_folder(input_image_folder, input_mask_folder, output_image_folder, output_mask_folder, label_format)


KeyboardInterrupt: 

In [15]:
import cv2
import numpy as np
import albumentations as A
import os
import json

def pca_color_augmentation(image, alpha_std=0.1):
    """ 使用PCA对图像进行颜色增强 """
    orig_img = image.astype(float)
    img_rs = orig_img.reshape(-1, 3)
    img_mean = np.mean(img_rs, axis=0)
    img_std = np.std(img_rs, axis=0)
    img_rs = (img_rs - img_mean) / img_std

    cov = np.cov(img_rs, rowvar=False)
    eigvals, eigvecs = np.linalg.eigh(cov)

    noise = np.random.normal(0, alpha_std, 3)
    noise = eigvecs @ (eigvals ** 0.5) * noise
    noise = (noise * img_std) + img_mean

    aug_img = orig_img + noise
    aug_img = np.clip(aug_img, 0, 255).astype(np.uint8)
    return aug_img

def augment_image(image, mask, n_augmentations=1):
    # 定义数据增强的变换序列
    transform = A.Compose([
        A.HorizontalFlip(p=0.5),  # 水平翻转，概率为50%
        A.VerticalFlip(p=0.5),    # 垂直翻转，概率为50%
        A.RandomRotate90(p=0.5),  # 随机旋转90度，概率为50%
        # A.Transpose(p=0.5),       # 转置操作（交换x和y轴），概率为50%
        A.RandomBrightnessContrast(p=0.2),  # 随机调整亮度和对比度，概率为20%
        A.ElasticTransform(p=0.2),  # 弹性变换，概率为20%
        A.GridDistortion(p=0.2),    # 网格扭曲，概率为20%
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=45, p=0.2),  # 平移、缩放和旋转，概率为20%
        # A.RandomCrop(height=256, width=256, p=1.0)  # 随机裁剪到256x256大小
    ])
    
    augmented_images = []
    augmented_masks = []
    
    for _ in range(n_augmentations):
        augmented = transform(image=image, mask=mask)
        image_aug = augmented['image']
        mask_aug = augmented['mask']
        
        # 应用PCA颜色增强
        image_aug = pca_color_augmentation(image_aug)
        
        augmented_images.append(image_aug)
        augmented_masks.append(mask_aug)
    
    return augmented_images, augmented_masks  # 返回增强后的图像和掩码

def read_mask(mask_path, label_format):
    # 读取掩码文件，根据标签格式选择不同的方法
    if label_format == "mask":
        mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)  # 读取PNG格式掩码
    elif label_format == "json":
        with open(mask_path, 'r') as f:
            annotations = json.load(f)  # 读取JSON文件
            mask = np.zeros((annotations['imageHeight'], annotations['imageWidth']), dtype=np.uint8)  # 创建空白掩码
            for shape in annotations['shapes']:
                points = np.array(shape['points'], dtype=np.int32)  # 将形状的点转换为数组
                cv2.fillPoly(mask, [points], color=(255))  # 用白色填充多边形
    else:
        raise ValueError("Unsupported label format. Use 'mask' or 'json'.")  # 抛出异常
    return mask  # 返回生成的掩码

def process_folder(input_image_folder, input_mask_folder, output_image_folder, output_mask_folder, label_format="mask", n_augmentations=5):
    # 检查输出文件夹是否存在，如果不存在则创建
    if not os.path.exists(output_image_folder):
        os.makedirs(output_image_folder)
    if not os.path.exists(output_mask_folder):
        os.makedirs(output_mask_folder)

    image_files = os.listdir(input_image_folder)  # 获取输入图像文件列表

    for image_file in image_files:
        image_path = os.path.join(input_image_folder, image_file)  # 构建图像文件路径
        
        # 根据标签格式选择相应的掩码文件名
        mask_file = image_file.replace('.jpg', '.png') if label_format == "mask" else image_file.replace('.jpg', '.json')
        mask_path = os.path.join(input_mask_folder, mask_file)  # 构建掩码文件路径
        
        if not os.path.exists(mask_path):
            print(f"Label file {mask_path} does not exist, skipping.")  # 如果掩码文件不存在，跳过该文件
            continue
        
        image = cv2.imread(image_path)  # 读取图像文件
        mask = read_mask(mask_path, label_format)  # 读取掩码文件
        
        augmented_images, augmented_masks = augment_image(image, mask, n_augmentations)  # 对图像和掩码进行数据增强
        
        for i, (image_aug, mask_aug) in enumerate(zip(augmented_images, augmented_masks)):
            output_image_path = os.path.join(output_image_folder, f"{os.path.splitext(image_file)[0]}_aug_{i}.jpg")  # 输出图像文件路径
            output_mask_path = os.path.join(output_mask_folder, f"{os.path.splitext(mask_file)[0]}_aug_{i}.png")  # 输出掩码文件路径
            
            cv2.imwrite(output_image_path, image_aug)  # 保存增强后的图像
            cv2.imwrite(output_mask_path, mask_aug)  # 保存增强后的掩码

if __name__ == "__main__":
    Dataset_Path = r"F:\work\dataset\rebar2D\train\label"  # 数据集路径
    img_path = r"F:\work\dataset\rebar2D\train\img"
    labelme_json_path = r"F:\work\dataset\rebar2D\train\label"
    mask_path=r"F:\work\dataset\rebar2D\train\mask" # 掩码图像文件夹路径
    
    # input_image_folder = img_path  # 输入图像文件夹路径
    input_image_folder = r"H:\data\rebar2D\train\img"
    input_mask_folder = mask_path  # 输入掩码文件夹路径
    output_image_folder = r"F:\work\dataset\rebar2D\train\img1"  # 输出图像文件夹路径
    output_mask_folder = r"F:\work\dataset\rebar2D\train\mask1"  # 输出掩码文件夹路径 

    label_format = "mask"  # 标签格式，可选"mask"或"json"
    n_augmentations = 5  # 每张图像增强的数量
    
    process_folder(input_image_folder, input_mask_folder, output_image_folder, output_mask_folder, label_format, n_augmentations)  # 处理文件夹中的文件


KeyboardInterrupt: 

In [17]:
import cv2  # 导入OpenCV库，用于图像处理
import numpy as np  # 导入NumPy库，用于数值计算
import albumentations as A  # 导入Albumentations库，用于图像增强
import os  # 导入OS库，用于文件和目录操作
import json  # 导入JSON库，用于处理JSON格式的数据

def pca_color_augmentation(image, alpha_std=0.1):
    """ 使用PCA对图像进行颜色增强 """
    orig_img = image.astype(float)  # 将图像转换为浮点数类型
    img_rs = orig_img.reshape(-1, 3)  # 将图像重塑为二维数组
    img_mean = np.mean(img_rs, axis=0)  # 计算每个通道的均值
    img_std = np.std(img_rs, axis=0)  # 计算每个通道的标准差
    img_rs = (img_rs - img_mean) / img_std  # 对图像进行标准化处理

    cov = np.cov(img_rs, rowvar=False)  # 计算协方差矩阵
    eigvals, eigvecs = np.linalg.eigh(cov)  # 计算特征值和特征向量

    noise = np.random.normal(0, alpha_std, 3)  # 生成正态分布的噪声
    noise = eigvecs @ (eigvals ** 0.5) * noise  # 应用PCA变换
    noise = (noise * img_std) + img_mean  # 反标准化处理

    aug_img = orig_img + noise  # 将噪声加到原图像上
    aug_img = np.clip(aug_img, 0, 255).astype(np.uint8)  # 裁剪值并转换为8位无符号整数
    return aug_img  # 返回增强后的图像

def augment_image(image, mask, n_augmentations=1):
    # 定义数据增强的变换序列
    transform = A.Compose([
        A.HorizontalFlip(p=0.5),  # 水平翻转，概率为50%
        A.VerticalFlip(p=0.5),    # 垂直翻转，概率为50%
        A.RandomRotate90(p=0.5),  # 随机旋转90度，概率为50%
        # A.Transpose(p=0.5),       # 转置操作（交换x和y轴），概率为50%
        A.RandomBrightnessContrast(p=0.2),  # 随机调整亮度和对比度，概率为20%
        A.ElasticTransform(p=0.2),  # 弹性变换，概率为20%
        A.GridDistortion(p=0.2),    # 网格扭曲，概率为20%
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=45, p=0.2),  # 平移、缩放和旋转，概率为20%
        # A.RandomCrop(height=256, width=256, p=1.0)  # 随机裁剪到256x256大小，概率为100%
    ])
    
    augmented_images = []  # 存储增强后的图像
    augmented_masks = []  # 存储增强后的掩码
    
    for _ in range(n_augmentations):
        augmented = transform(image=image, mask=mask)  # 对图像和掩码进行增强
        image_aug = augmented['image']  # 获取增强后的图像
        mask_aug = augmented['mask']  # 获取增强后的掩码
        
        # 应用PCA颜色增强
        image_aug = pca_color_augmentation(image_aug)
        
        augmented_images.append(image_aug)  # 添加到增强图像列表
        augmented_masks.append(mask_aug)  # 添加到增强掩码列表
    
    return augmented_images, augmented_masks  # 返回增强后的图像和掩码

def read_mask(mask_path, label_format):
    # 读取掩码文件，根据标签格式选择不同的方法
    if label_format == "mask":
        mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)  # 读取PNG格式掩码
    elif label_format == "json":
        with open(mask_path, 'r') as f:
            annotations = json.load(f)  # 读取JSON文件
            mask = np.zeros((annotations['imageHeight'], annotations['imageWidth']), dtype=np.uint8)  # 创建空白掩码
            for shape in annotations['shapes']:
                points = np.array(shape['points'], dtype=np.int32)  # 将形状的点转换为数组
                cv2.fillPoly(mask, [points], color=(255))  # 用白色填充多边形
    else:
        raise ValueError("Unsupported label format. Use 'mask' or 'json'.")  # 抛出异常
    return mask  # 返回生成的掩码

def process_folder(input_image_folder, input_mask_folder, output_image_folder, output_mask_folder, label_format="mask", n_augmentations=5):
    # 检查输出文件夹是否存在，如果不存在则创建
    if not os.path.exists(output_image_folder):
        os.makedirs(output_image_folder)
    if not os.path.exists(output_mask_folder):
        os.makedirs(output_mask_folder)

    image_files = os.listdir(input_image_folder)  # 获取输入图像文件列表

    for image_file in image_files:
        image_path = os.path.join(input_image_folder, image_file)  # 构建图像文件路径
        
        # 根据标签格式选择相应的掩码文件名
        if image_path.endswith('.jpg'):
            mask_file = image_file.replace('.jpg', '.png') if label_format == "mask" else image_file.replace(
            '.jpg','.json')
        elif image_path.endswith('.bmp'):
            mask_file = image_file.replace('.bmp', '.png') if label_format == "mask" else image_file.replace(
                '.bmp','.json')
        else:
            print(f"Unsupported image format for {image_path}, skipping.")
        mask_path = os.path.join(input_mask_folder, mask_file)  # 构建掩码文件路径
        
        if not os.path.exists(mask_path):
            print(f"Label file {mask_path} does not exist, skipping.")  # 如果掩码文件不存在，跳过该文件
            continue
        
        image = cv2.imread(image_path)  # 读取图像文件
        mask = read_mask(mask_path, label_format)  # 读取掩码文件
        
        augmented_images, augmented_masks = augment_image(image, mask, n_augmentations)  # 对图像和掩码进行数据增强
        
        for i, (image_aug, mask_aug) in enumerate(zip(augmented_images, augmented_masks)):
            output_image_path = os.path.join(output_image_folder, f"{os.path.splitext(image_file)[0]}_aug_{i}.jpg")  # 输出图像文件路径
            output_mask_path = os.path.join(output_mask_folder, f"{os.path.splitext(mask_file)[0]}_aug_{i}.png")  # 输出掩码文件路径
            
            cv2.imwrite(output_image_path, image_aug)  # 保存增强后的图像
            cv2.imwrite(output_mask_path, mask_aug)  # 保存增强后的掩码

if __name__ == "__main__":

    # input_image_folder = r"F:\work\dataset\rebar2D\train\img"  # 输入图像文件夹路径
    input_image_folder = r"H:\data\rebar2D\train\img"
    input_mask_folder =r"F:\work\dataset\rebar2D\train\mask"  # 输入掩码文件夹路径
    output_image_folder = r"F:\work\dataset\rebar2D\train\img1"  # 输出图像文件夹路径
    output_mask_folder = r"F:\work\dataset\rebar2D\train\mask1"  # 输出掩码文件夹路径
    
    label_format = "mask"  # 标签格式，可选"mask"或"json"
    
    n_augmentations = 5  # 每张图像增强的数量
    
    process_folder(input_image_folder, input_mask_folder, output_image_folder, output_mask_folder, label_format, n_augmentations)  # 处理文件夹中的文件


KeyboardInterrupt: 

In [24]:
import cv2
import numpy as np

# 读取图像
image = cv2.imread(r'F:\work\python\clone\dataset\rebar2d\images\Image_20240622152006003.jpg')

# 将图像从BGR颜色空间转换为HSV颜色空间
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# 定义棕褐色的HSV范围
brown_lower = np.array([10, 50, 20])
brown_upper = np.array([30, 255, 200])

# 创建掩膜，仅保留棕褐色区域
brown_mask = cv2.inRange(hsv_image, brown_lower, brown_upper)

# 对原始图像应用掩膜
enhanced_image = cv2.bitwise_and(image, image, mask=brown_mask)

# 增强棕褐色区域的亮度和饱和度
enhanced_hsv_image = hsv_image.copy()
enhanced_hsv_image[..., 2] = cv2.add(enhanced_hsv_image[..., 2], 50)  # 调整亮度
enhanced_hsv_image[..., 1] = cv2.add(enhanced_hsv_image[..., 1], 50)  # 调整饱和度

# 将调整后的HSV图像转换回BGR颜色空间
enhanced_bgr_image = cv2.cvtColor(enhanced_hsv_image, cv2.COLOR_HSV2BGR)

# 将增强后的棕褐色区域与原始图像合并
final_image = cv2.addWeighted(image, 0.7, enhanced_bgr_image, 0.3, 0)

# 显示结果
cv2.imwrite(r"F:\work\dataset\rebar2D\train\TEMP\11.jpg", final_image)



True

In [25]:
import cv2  # 导入OpenCV库，用于图像处理
import numpy as np  # 导入NumPy库，用于数值计算
import albumentations as A  # 导入Albumentations库，用于图像增强
import os  # 导入OS库，用于文件和目录操作
import json  # 导入JSON库，用于处理JSON格式的数据

def pca_color_augmentation(image, alpha_std=0.1):
    """ 使用PCA对图像进行颜色增强 """
    orig_img = image.astype(float)  # 将图像转换为浮点数类型
    img_rs = orig_img.reshape(-1, 3)  # 将图像重塑为二维数组
    img_mean = np.mean(img_rs, axis=0)  # 计算每个通道的均值
    img_std = np.std(img_rs, axis=0)  # 计算每个通道的标准差
    img_rs = (img_rs - img_mean) / img_std  # 对图像进行标准化处理

    cov = np.cov(img_rs, rowvar=False)  # 计算协方差矩阵
    eigvals, eigvecs = np.linalg.eigh(cov)  # 计算特征值和特征向量

    noise = np.random.normal(0, alpha_std, 3)  # 生成正态分布的噪声
    noise = eigvecs @ (eigvals ** 0.5) * noise  # 应用PCA变换
    noise = (noise * img_std) + img_mean  # 反标准化处理

    aug_img = orig_img + noise  # 将噪声加到原图像上
    aug_img = np.clip(aug_img, 0, 255).astype(np.uint8)  # 裁剪值并转换为8位无符号整数
    return aug_img  # 返回增强后的图像

def brownify_augmentation(image, alpha=0.5):
    """ 对棕褐色物品进行颜色增强 """
    brown_tint = np.array([19, 69, 139], dtype=np.float32)  # 定义棕褐色的色调
    image = image.astype(np.float32)  # 将图像转换为浮点数类型
    image = (1 - alpha) * image + alpha * brown_tint  # 混合原图像和棕褐色色调
    image = np.clip(image, 0, 255).astype(np.uint8)  # 裁剪值并转换为8位无符号整数
    return image  # 返回增强后的图像

def augment_image(image, mask, n_augmentations=1, color_augmentation='pca'):
    # 定义数据增强的变换序列
    transform = A.Compose([
        A.HorizontalFlip(p=0.5),  # 水平翻转，概率为50%
        A.VerticalFlip(p=0.5),    # 垂直翻转，概率为50%
        A.RandomRotate90(p=0.5),  # 随机旋转90度，概率为50%
        # A.Transpose(p=0.5),       # 转置操作（交换x和y轴），概率为50%
        A.RandomBrightnessContrast(p=0.2),  # 随机调整亮度和对比度，概率为20%
        A.ElasticTransform(p=0.2),  # 弹性变换，概率为20%
        A.GridDistortion(p=0.2),    # 网格扭曲，概率为20%
        A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=45, p=0.2),  # 平移、缩放和旋转，概率为20%
        # A.RandomCrop(height=256, width=256, p=1.0)  # 随机裁剪到256x256大小，概率为100%
    ])
    
    augmented_images = []  # 存储增强后的图像
    augmented_masks = []  # 存储增强后的掩码
    
    for _ in range(n_augmentations):
        augmented = transform(image=image, mask=mask)  # 对图像和掩码进行增强
        image_aug = augmented['image']  # 获取增强后的图像
        mask_aug = augmented['mask']  # 获取增强后的掩码
        
        # 应用选择的颜色增强方法
        if color_augmentation == 'pca':
            image_aug = pca_color_augmentation(image_aug)
        elif color_augmentation == 'brownify':
            image_aug = brownify_augmentation(image_aug)
        
        augmented_images.append(image_aug)  # 添加到增强图像列表
        augmented_masks.append(mask_aug)  # 添加到增强掩码列表
    
    return augmented_images, augmented_masks  # 返回增强后的图像和掩码

def read_mask(mask_path, label_format):
    # 读取掩码文件，根据标签格式选择不同的方法
    if label_format == "mask":
        mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)  # 读取PNG格式掩码
    elif label_format == "json":
        with open(mask_path, 'r') as f:
            annotations = json.load(f)  # 读取JSON文件
            mask = np.zeros((annotations['imageHeight'], annotations['imageWidth']), dtype=np.uint8)  # 创建空白掩码
            for shape in annotations['shapes']:
                points = np.array(shape['points'], dtype=np.int32)  # 将形状的点转换为数组
                cv2.fillPoly(mask, [points], color=(255))  # 用白色填充多边形
    else:
        raise ValueError("Unsupported label format. Use 'mask' or 'json'.")  # 抛出异常
    return mask  # 返回生成的掩码

def process_folder(input_image_folder, input_mask_folder, output_image_folder, output_mask_folder, label_format="mask", n_augmentations=5, color_augmentation='pca'):
    # 检查输出文件夹是否存在，如果不存在则创建
    if not os.path.exists(output_image_folder):
        os.makedirs(output_image_folder)
    if not os.path.exists(output_mask_folder):
        os.makedirs(output_mask_folder)

    image_files = os.listdir(input_image_folder)  # 获取输入图像文件列表

    for image_file in image_files:
        image_path = os.path.join(input_image_folder, image_file)  # 构建图像文件路径
        
        # 根据标签格式选择相应的掩码文件名
        if image_path.endswith('.jpg'):
            mask_file = image_file.replace('.jpg', '.png') if label_format == "mask" else image_file.replace(
                '.jpg', '.json')
        elif image_path.endswith('.bmp'):
            mask_file = image_file.replace('.bmp', '.png') if label_format == "mask" else image_file.replace(
                '.bmp', '.json')
        else:
            print(f"Unsupported image format for {image_path}, skipping.")
        
        if not os.path.exists(mask_path):
            print(f"Label file {mask_path} does not exist, skipping.")  # 如果掩码文件不存在，跳过该文件
            continue
        
        image = cv2.imread(image_path)  # 读取图像文件
        mask = read_mask(mask_path, label_format)  # 读取掩码文件
        
        augmented_images, augmented_masks = augment_image(image, mask, n_augmentations, color_augmentation)  # 对图像和掩码进行数据增强
        
        for i, (image_aug, mask_aug) in enumerate(zip(augmented_images, augmented_masks)):
            output_image_path = os.path.join(output_image_folder, f"{os.path.splitext(image_file)[0]}_aug_{i}.jpg")  # 输出图像文件路径
            output_mask_path = os.path.join(output_mask_folder, f"{os.path.splitext(mask_file)[0]}_aug_{i}.png")  # 输出掩码文件路径
            
            cv2.imwrite(output_image_path, image_aug)  # 保存增强后的图像
            cv2.imwrite(output_mask_path, mask_aug)  # 保存增强后的掩码

if __name__ == "__main__":
    input_image_folder = r"H:\data\rebar2D\train\img"
    input_mask_folder = r"F:\work\dataset\rebar2D\train\mask"  # 输入掩码文件夹路径
    output_image_folder = r"F:\work\dataset\rebar2D\train\img1"  # 输出图像文件夹路径
    output_mask_folder = r"F:\work\dataset\rebar2D\train\mask1"  # 输出掩码文件夹路径
    label_format = "mask"  # 标签格式，可选"mask"或"json"
    n_augmentations = 5  # 每张图像增强的数量
    color_augmentation = 'brownify'  # 颜色增强方式，可选"pca"或"brownify"
    
    process_folder(input_image_folder, input_mask_folder, output_image_folder, output_mask_folder, label_format, n_augmentations, color_augmentation)  # 处理文件夹中的文件


TypeError: mask must be numpy array type

In [ ]:
import os

def process_files_in_directory(directory_path, file_extension=None):
    """
    对目录中的所有文件进行处理
    :param directory_path: 目录路径
    :param file_extension: 只处理指定扩展名的文件（例如".txt"），默认处理所有文件
    """
    if not os.path.isdir(directory_path):
        print(f"{directory_path} is not a valid directory.")
        return
    
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            if file_extension is None or file.endswith(file_extension):
                file_path = os.path.join(root, file)
                process_single_file(file_path)

def process_single_file(file_path):
    # 在这里处理单个文件的逻辑
    print(f"Processing file: {file_path}")
    # 假设你的处理逻辑是读取文件并打印其内容
    with open(file_path, 'r') as file:
        content = file.read()
        print(content)

# 使用示例
directory_to_process = '/path/to/your/directory'
process_files_in_directory(directory_to_process, file_extension=".txt")


In [26]:
from PIL import Image, ImageOps
import os

def apply_mask(original_image_path, mask_image_path, output_image_path):
    # 打开原图和掩码图像
    original_image = Image.open(original_image_path).convert("RGBA")
    mask_image = Image.open(mask_image_path).convert("L")  # 转换为灰度图像

    # 创建彩色版的掩码图像（例如将其变为红色）
    color_mask = ImageOps.colorize(mask_image, black=(0, 0, 0, 0), white=(255, 0, 0, 128))

    # 将彩色掩码图像叠加到原图上
    combined_image = Image.alpha_composite(original_image, color_mask)

    # 保存生成的图像
    combined_image.save(output_image_path)

def batch_process_images(original_folder, mask_folder, output_folder):
    # 确保输出文件夹存在
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # 获取原图和掩码图像文件名
    original_images = [f for f in os.listdir(original_folder) if f.endswith(('jpg', 'bmp'))]
    mask_images = [f for f in os.listdir(mask_folder) if f.endswith('png')]

    for original_image_name in original_images:
        # 对应的掩码图像文件名
        mask_image_name = original_image_name.rsplit('.', 1)[0] + '.png'

        if mask_image_name in mask_images:
            original_image_path = os.path.join(original_folder, original_image_name)
            mask_image_path = os.path.join(mask_folder, mask_image_name)
            output_image_path = os.path.join(output_folder, original_image_name)

            # 应用掩码
            apply_mask(original_image_path, mask_image_path, output_image_path)
            print(f'Processed {original_image_name} with mask {mask_image_name}')

if __name__ == "__main__":
    original_folder = r"F:\work\dataset\rebar2D\train\img"
    mask_folder = r"F:\work\dataset\rebar2D\train\mask"
    output_folder = r"F:\work\dataset\rebar2D\train\TEMP"

    batch_process_images(original_folder, mask_folder, output_folder)


ValueError: images do not match

In [6]:
import cv2
import numpy as np
from ultralytics import YOLO

# 使用YOLOv8模型检测钢筋并分割
def detect_rebar(image_path):
    model = YOLO(r"F:\work\python\clone\2d\ultralnew\ultralytics\best1.pt")  # 替换为你的YOLOv8分割模型路径
    results = model(image_path)

    # 获取分割结果的边界框
    boxes = results[0].boxes.cpu().numpy()  # 假设返回的第一个结果包含边界框

    return boxes

# 获取边界框四个点并扩大
def expand_bbox(bbox, expansion_factor=0.1):
    x1, y1, x2, y2 = bbox
    width = x2 - x1
    height = y2 - y1

    x1_exp = max(0, x1 - width * expansion_factor)
    y1_exp = max(0, y1 - height * expansion_factor)
    x2_exp = min(image.shape[1], x2 + width * expansion_factor)
    y2_exp = min(image.shape[0], y2 + height * expansion_factor)

    return int(x1_exp), int(y1_exp), int(x2_exp), int(y2_exp)

# 提取扩大后的边界框内容
def extract_expanded_bbox_content(image, bbox):
    x1, y1, x2, y2 = bbox
    return image[y1:y2, x1:x2]

# 使用传统方法获得钢筋的轮廓
def get_rebar_contour(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresh = cv2.threshold(blurred, 60, 255, cv2.THRESH_BINARY_INV)

    # 找到轮廓
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

# 将轮廓写入和原图同大小的画布中并保存
def draw_contours_on_canvas(image, contours, bbox, output_path):
    canvas = np.zeros_like(image)
    x1, y1, x2, y2 = bbox
    for contour in contours:
        contour += np.array([x1, y1])
        cv2.drawContours(canvas, [contour], -1, (255, 255, 255), thickness=cv2.FILLED)
    cv2.imwrite(output_path, canvas)

if __name__ == '__main__':
    image_path = r"F:\work\dataset\rebar2D\train\img\Image_20240622155643472.jpg"  # 输入图像路径
    output_path = r"F:\work\dataset\rebar2D\train\TEMP\test.jpg"  # 输出图像路径

    # 读取图像
    image = cv2.imread(image_path)

    # 检测钢筋并获取边界框
    boxes = detect_rebar(image_path)

    # 假设只有一个钢筋对象，获取其边界框并扩大
    bbox = boxes[0].xyxy[0]
    expanded_bbox = expand_bbox(bbox)

    # 提取扩大后的边界框内容
    expanded_bbox_content = extract_expanded_bbox_content(image, expanded_bbox)

    # 获取钢筋的轮廓
    contours = get_rebar_contour(expanded_bbox_content)

    # 将轮廓写入和原图同大小的画布中并保存
    draw_contours_on_canvas(image, contours, expanded_bbox, output_path)
    print(f'Result saved to {output_path}')



image 1/1 F:\work\dataset\rebar2D\train\img\Image_20240622155643472.jpg: 448x640 1 rebar, 31.0ms
Speed: 1.5ms preprocess, 31.0ms inference, 2.2ms postprocess per image at shape (1, 3, 448, 640)
Result saved to F:\work\dataset\rebar2D\train\TEMP\test.jpg


In [1]:
import sys
import cv2
import numpy as np
from ctypes import *

# 添加 MVS SDK 的 Python 包路径
sys.path.append(r"F:\install\MVS\Development\Samples\Python\MvImport")

# 导入 MVS SDK
from MvCameraControl_class import *

def main():
    # 创建相机对象
    cam = MvCamera()

    # 设备信息列表
    deviceList = MV_CC_DEVICE_INFO_LIST()
    tlayerType = MV_GIGE_DEVICE | MV_USB_DEVICE

    # 枚举设备
    ret = cam.MV_CC_EnumDevices(tlayerType, deviceList)
    if ret != 0:
        print(f"Enum devices failed! ret[0x{ret:x}]")
        return

    if deviceList.nDeviceNum == 0:
        print("No devices found!")
        return

    # 选择第一个设备
    stDeviceList = cast(deviceList.pDeviceInfo[0], POINTER(MV_CC_DEVICE_INFO)).contents

    # 创建句柄
    ret = cam.MV_CC_CreateHandle(stDeviceList)
    if ret != 0:
        print(f"Create handle failed! ret[0x{ret:x}]")
        return

    # 打开设备
    ret = cam.MV_CC_OpenDevice(MV_ACCESS_Exclusive, 0)
    if ret != 0:
        print(f"Open device failed! ret[0x{ret:x}]")
        return

    print("Device opened successfully")

    # 获取并打印当前分辨率
    width = MVCC_INTVALUE()
    height = MVCC_INTVALUE()
    ret = cam.MV_CC_GetIntValue("Width", width)
    if ret != 0:
        print(f"Get width failed! ret[0x{ret:x}]")
        return
    ret = cam.MV_CC_GetIntValue("Height", height)
    if ret != 0:
        print(f"Get height failed! ret[0x{ret:x}]")
        return

    print(f"Current resolution: Width = {width.nCurValue}, Height = {height.nCurValue}")

    # 获取并打印当前像素格式
    stEnumValue = MVCC_ENUMVALUE()
    ret = cam.MV_CC_GetEnumValue("PixelFormat", stEnumValue)
    if ret != 0:
        print(f"Get pixel format failed! ret[0x{ret:x}]")
        return

    print(f"Current pixel format: 0x{stEnumValue.nCurValue:x}")

    # 设置新的分辨率和像素格式（根据需要进行调整）
    new_width = 1920
    new_height = 1080
    new_pixel_format = 0x110000d  # 替换为你想使用的像素格式

    ret = cam.MV_CC_SetIntValue("Width", new_width)
    if ret != 0:
        print(f"Set width failed! ret[0x{ret:x}]")
        return

    ret = cam.MV_CC_SetIntValue("Height", new_height)
    if ret != 0:
        print(f"Set height failed! ret[0x{ret:x}]")
        return

    ret = cam.MV_CC_SetEnumValue("PixelFormat", new_pixel_format)
    if ret != 0:
        print(f"Set pixel format failed! ret[0x{ret:x}]")
        return

    print("Resolution and pixel format set successfully")

    # 开始采集
    ret = cam.MV_CC_StartGrabbing()
    if ret != 0:
        print(f"Start grabbing failed! ret[0x{ret:x}]")
        return

    data_buf = None
    buf_size = new_width * new_height * 3  # RGB8 格式缓冲区大小
    while True:
        stFrameInfo = MV_FRAME_OUT_INFO_EX()
        data_buf = (c_ubyte * buf_size)()
        ret = cam.MV_CC_GetOneFrameTimeout(data_buf, buf_size, stFrameInfo, 1000)
        if ret == 0:
            print(f"Get one frame: Width[{stFrameInfo.nWidth}], Height[{stFrameInfo.nHeight}], FrameNum[{stFrameInfo.nFrameNum}]")
            img_buff = (c_ubyte * stFrameInfo.nFrameLen).from_address(addressof(data_buf))
            img = np.frombuffer(img_buff, dtype=np.uint8)

            # 确定图像形状，根据像素格式调整通道数
            if new_pixel_format == 0x110000d:  # RGB8
                img = img.reshape((stFrameInfo.nHeight, stFrameInfo.nWidth, 3))
            elif new_pixel_format == 0x1080009:  # Mono8
                img = img.reshape((stFrameInfo.nHeight, stFrameInfo.nWidth))
            elif new_pixel_format in {0x10c0027, 0x10c002b}:  # BayerRG8, BayerGB8
                img = img.reshape((stFrameInfo.nHeight, stFrameInfo.nWidth))
                img = cv2.cvtColor(img, cv2.COLOR_BAYER_RG2RGB if new_pixel_format == 0x10c0027 else cv2.COLOR_BAYER_BG2RGB)
            elif new_pixel_format == 0x1100011:  # YUV422Packed
                img = img.reshape((stFrameInfo.nHeight, stFrameInfo.nWidth, 2))
                img = cv2.cvtColor(img, cv2.COLOR_YUV2BGR_YUYV)
            else:
                print("Unsupported pixel format")
                break

            # 显示图像
            cv2.imshow('Camera', img)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            print(f"Get image failed! ret[0x{ret:x}]")
            break

    # 停止采集
    cam.MV_CC_StopGrabbing()
    cam.MV_CC_CloseDevice()
    cam.MV_CC_DestroyHandle()

    # 释放资源
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Device opened successfully
Current resolution: Width = 3072, Height = 2048
Current pixel format: 0x110000d
Resolution and pixel format set successfully
Get one frame: Width[1920], Height[1080], FrameNum[1]


ValueError: cannot reshape array of size 4147200 into shape (1080,1920,3)

In [1]:
import sys
import cv2
import numpy as np
from ctypes import *

# 添加 MVS SDK 的 Python 包路径
sys.path.append(r"F:\install\MVS\Development\Samples\Python\MvImport")

# 导入 MVS SDK
from MvCameraControl_class import *

def main():
    # 创建相机对象
    cam = MvCamera()

    # 设备信息列表
    deviceList = MV_CC_DEVICE_INFO_LIST()
    tlayerType = MV_GIGE_DEVICE | MV_USB_DEVICE

    # 枚举设备
    ret = cam.MV_CC_EnumDevices(tlayerType, deviceList)
    if ret != 0:
        print(f"Enum devices failed! ret[0x{ret:x}]")
        return

    if deviceList.nDeviceNum == 0:
        print("No devices found!")
        return

    # 选择第一个设备
    stDeviceList = cast(deviceList.pDeviceInfo[0], POINTER(MV_CC_DEVICE_INFO)).contents

    # 创建句柄
    ret = cam.MV_CC_CreateHandle(stDeviceList)
    if ret != 0:
        print(f"Create handle failed! ret[0x{ret:x}]")
        return

    # 打开设备
    ret = cam.MV_CC_OpenDevice(MV_ACCESS_Exclusive, 0)
    if ret != 0:
        print(f"Open device failed! ret[0x{ret:x}]")
        return

    print("Device opened successfully")

    # 获取并打印当前分辨率
    width = MVCC_INTVALUE()
    height = MVCC_INTVALUE()
    ret = cam.MV_CC_GetIntValue("Width", width)
    if ret != 0:
        print(f"Get width failed! ret[0x{ret:x}]")
        return
    ret = cam.MV_CC_GetIntValue("Height", height)
    if ret != 0:
        print(f"Get height failed! ret[0x{ret:x}]")
        return

    print(f"Current resolution: Width = {width.nCurValue}, Height = {height.nCurValue}")

    # 获取并打印当前像素格式
    stEnumValue = MVCC_ENUMVALUE()
    ret = cam.MV_CC_GetEnumValue("PixelFormat", stEnumValue)
    if ret != 0:
        print(f"Get pixel format failed! ret[0x{ret:x}]")
        return

    print(f"Current pixel format: 0x{stEnumValue.nCurValue:x}")

    # 设置新的分辨率和像素格式（根据需要进行调整）
    new_width = 1920
    new_height = 1080
    new_pixel_format = 0x110000d  # 替换为你想使用的像素格式

    ret = cam.MV_CC_SetIntValue("Width", new_width)
    if ret != 0:
        print(f"Set width failed! ret[0x{ret:x}]")
        return

    ret = cam.MV_CC_SetIntValue("Height", new_height)
    if ret != 0:
        print(f"Set height failed! ret[0x{ret:x}]")
        return

    ret = cam.MV_CC_SetEnumValue("PixelFormat", new_pixel_format)
    if ret != 0:
        print(f"Set pixel format failed! ret[0x{ret:x}]")
        return

    print("Resolution and pixel format set successfully")

    # 开始采集
    ret = cam.MV_CC_StartGrabbing()
    if ret != 0:
        print(f"Start grabbing failed! ret[0x{ret:x}]")
        return

    data_buf = None
    buf_size = new_width * new_height * 3  # RGB8 格式缓冲区大小
    while True:
        stFrameInfo = MV_FRAME_OUT_INFO_EX()
        data_buf = (c_ubyte * buf_size)()
        ret = cam.MV_CC_GetOneFrameTimeout(data_buf, buf_size, stFrameInfo, 1000)
        if ret == 0:
            print(f"Get one frame: Width[{stFrameInfo.nWidth}], Height[{stFrameInfo.nHeight}], FrameNum[{stFrameInfo.nFrameNum}]")
            print(f"Frame length: {stFrameInfo.nFrameLen}")
            img_buff = (c_ubyte * stFrameInfo.nFrameLen).from_address(addressof(data_buf))
            img = np.frombuffer(img_buff, dtype=np.uint8)

            print(f"Image buffer size: {img.size}")
            print(f"Expected buffer size: {new_width * new_height * 3}")

            # 确定图像形状，根据像素格式调整通道数
            if new_pixel_format == 0x110000d:  # RGB8
                img = img.reshape((stFrameInfo.nHeight, stFrameInfo.nWidth, 3))
            elif new_pixel_format == 0x1080009:  # Mono8
                img = img.reshape((stFrameInfo.nHeight, stFrameInfo.nWidth))
            elif new_pixel_format in {0x10c0027, 0x10c002b}:  # BayerRG8, BayerGB8
                img = img.reshape((stFrameInfo.nHeight, stFrameInfo.nWidth))
                img = cv2.cvtColor(img, cv2.COLOR_BAYER_RG2RGB if new_pixel_format == 0x10c0027 else cv2.COLOR_BAYER_BG2RGB)
            elif new_pixel_format == 0x1100011:  # YUV422Packed
                img = img.reshape((stFrameInfo.nHeight, stFrameInfo.nWidth, 2))
                img = cv2.cvtColor(img, cv2.COLOR_YUV2BGR_YUYV)
            else:
                print("Unsupported pixel format")
                break

            # 显示图像
            cv2.imshow('Camera', img)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            print(f"Get image failed! ret[0x{ret:x}]")
            break

    # 停止采集
    cam.MV_CC_StopGrabbing()
    cam.MV_CC_CloseDevice()
    cam.MV_CC_DestroyHandle()

    # 释放资源
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Device opened successfully
Current resolution: Width = 1920, Height = 1080
Current pixel format: 0x110000d
Resolution and pixel format set successfully
Get one frame: Width[1920], Height[1080], FrameNum[1649]
Frame length: 4147200
Image buffer size: 4147200
Expected buffer size: 6220800


ValueError: cannot reshape array of size 4147200 into shape (1080,1920,3)

In [1]:
import sys
import cv2
import numpy as np
from ctypes import *

# 添加 MVS SDK 的 Python 包路径
sys.path.append(r"F:\install\MVS\Development\Samples\Python\MvImport")

# 导入 MVS SDK
from MvCameraControl_class import *

def main():
    # 创建相机对象
    cam = MvCamera()

    # 设备信息列表
    deviceList = MV_CC_DEVICE_INFO_LIST()
    tlayerType = MV_GIGE_DEVICE | MV_USB_DEVICE

    # 枚举设备
    ret = cam.MV_CC_EnumDevices(tlayerType, deviceList)
    if ret != 0:
        print(f"Enum devices failed! ret[0x{ret:x}]")
        return

    if deviceList.nDeviceNum == 0:
        print("No devices found!")
        return

    # 选择第一个设备
    stDeviceList = cast(deviceList.pDeviceInfo[0], POINTER(MV_CC_DEVICE_INFO)).contents

    # 创建句柄
    ret = cam.MV_CC_CreateHandle(stDeviceList)
    if ret != 0:
        print(f"Create handle failed! ret[0x{ret:x}]")
        return

    # 打开设备
    ret = cam.MV_CC_OpenDevice(MV_ACCESS_Exclusive, 0)
    if ret != 0:
        print(f"Open device failed! ret[0x{ret:x}]")
        return

    print("Device opened successfully")

    # 获取并打印当前分辨率
    width = MVCC_INTVALUE()
    height = MVCC_INTVALUE()
    ret = cam.MV_CC_GetIntValue("Width", width)
    if ret != 0:
        print(f"Get width failed! ret[0x{ret:x}]")
        return
    ret = cam.MV_CC_GetIntValue("Height", height)
    if ret != 0:
        print(f"Get height failed! ret[0x{ret:x}]")
        return

    print(f"Current resolution: Width = {width.nCurValue}, Height = {height.nCurValue}")

    # 获取并打印当前像素格式
    stEnumValue = MVCC_ENUMVALUE()
    ret = cam.MV_CC_GetEnumValue("PixelFormat", stEnumValue)
    if ret != 0:
        print(f"Get pixel format failed! ret[0x{ret:x}]")
        return

    print(f"Current pixel format: 0x{stEnumValue.nCurValue:x}")

    # 设置新的分辨率和像素格式（根据需要进行调整）
    new_width = 3072
    new_height = 2048
    new_pixel_format = 0x110000d  # 替换为你想使用的像素格式

    ret = cam.MV_CC_SetIntValue("Width", new_width)
    if ret != 0:
        print(f"Set width failed! ret[0x{ret:x}]")
        return

    ret = cam.MV_CC_SetIntValue("Height", new_height)
    if ret != 0:
        print(f"Set height failed! ret[0x{ret:x}]")
        return

    ret = cam.MV_CC_SetEnumValue("PixelFormat", new_pixel_format)
    if ret != 0:
        print(f"Set pixel format failed! ret[0x{ret:x}]")
        return

    print("Resolution and pixel format set successfully")

    # 开始采集
    ret = cam.MV_CC_StartGrabbing()
    if ret != 0:
        print(f"Start grabbing failed! ret[0x{ret:x}]")
        return

    data_buf = None
    buf_size = new_width * new_height * 3  # RGB8 格式缓冲区大小
    while True:
        stFrameInfo = MV_FRAME_OUT_INFO_EX()
        data_buf = (c_ubyte * buf_size)()
        ret = cam.MV_CC_GetOneFrameTimeout(data_buf, buf_size, stFrameInfo, 1000)
        if ret == 0:
            print(f"Get one frame: Width[{stFrameInfo.nWidth}], Height[{stFrameInfo.nHeight}], FrameNum[{stFrameInfo.nFrameNum}]")
            print(f"Frame length: {stFrameInfo.nFrameLen}")
            img_buff = (c_ubyte * stFrameInfo.nFrameLen).from_address(addressof(data_buf))
            img = np.frombuffer(img_buff, dtype=np.uint8)

            print(f"Image buffer size: {img.size}")
            print(f"Expected buffer size: {new_width * new_height * 3}")

            # 确定图像形状，根据像素格式调整通道数
            if new_pixel_format == 0x110000d:  # RGB8
                img = img.reshape((stFrameInfo.nHeight, stFrameInfo.nWidth, 3))
            elif new_pixel_format == 0x1080009:  # Mono8
                img = img.reshape((stFrameInfo.nHeight, stFrameInfo.nWidth))
            elif new_pixel_format in {0x10c0027, 0x10c002b}:  # BayerRG8, BayerGB8
                img = img.reshape((stFrameInfo.nHeight, stFrameInfo.nWidth))
                img = cv2.cvtColor(img, cv2.COLOR_BAYER_RG2RGB if new_pixel_format == 0x10c0027 else cv2.COLOR_BAYER_BG2RGB)
            elif new_pixel_format == 0x1100011:  # YUV422Packed
                img = img.reshape((stFrameInfo.nHeight, stFrameInfo.nWidth, 2))
                img = cv2.cvtColor(img, cv2.COLOR_YUV2BGR_YUYV)
            else:
                print("Unsupported pixel format")
                break

            # 显示图像
            cv2.imshow('Camera', img)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            print(f"Get image failed! ret[0x{ret:x}]")
            break

    # 停止采集
    cam.MV_CC_StopGrabbing()
    cam.MV_CC_CloseDevice()
    cam.MV_CC_DestroyHandle()

    # 释放资源
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Device opened successfully
Current resolution: Width = 1920, Height = 1080
Current pixel format: 0x110000d
Resolution and pixel format set successfully
Get one frame: Width[3072], Height[2048], FrameNum[2410]
Frame length: 12582912
Image buffer size: 12582912
Expected buffer size: 18874368


ValueError: cannot reshape array of size 12582912 into shape (2048,3072,3)